In [ ]:
import os, psycopg2, folium, pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

# db connection

In [ ]:
from sqlalchemy import create_engine 
from sqlalchemy.orm import sessionmaker
# database credentials
db_usr, db_pwd = '', '' # your database user name and password

# database login
host, port, db = 'nc-health-data-prod.cluster-ccsgl7rk4urn.eu-central-1.rds.amazonaws.com', 5432, 'master'
engine = create_engine('postgresql://'+db_usr+':'+db_pwd+'@'+host+':'+str(port)+'/'+db)
Session = sessionmaker(bind=engine)
session = Session()
conn = engine.connect()

In [ ]:
# db connection for queries without output
conn_exe = psycopg2.connect('host='+host+' port='+str(port)+' dbname='+db+' user='+db_usr+' password='+db_pwd)
cursor = conn_exe.cursor()

In [ ]:
# close db connection
conn_exe.close()

# data

## identify mass event visitor dids and their home locations

osm_id=  -9207449

Use the plz to find the tiles and make the search faster

tile_ids 61075 61076

## If osm_id is osm_dt_buildings_tiles

In [ ]:
query_osmid_did = f"""
    drop table if exists euro_stuttgart;
    create table euro_stuttgart as (
    with t1 as (
    	select ex."day", ex.did, ex.tile_id, ex.tile_arr--unnest(ex.tile_arr) as tile8
    	from ex_corona_sdkv6_2024_27 as ex
        where day = '2024-07-05'
    )
    , t2 as (
    	select tile_id, osm_id
    	from osm_dt_buildings_tiles
    )
    , t3 as (
    	select *
    	from t2
    	where osm_id = -9207449
    )
    , t4 as (
    select 
        t1."day", 
        t1.did, 
        t1.tile_id, 
        unnest(t1.tile_arr) as tile8, 
        t3.tile_id as tile, 
        t3.osm_id
    from t1
    join t3
    on t1.tile_id = t3.tile_id
    )
    select distinct 
        t4."day", 
        t4.osm_id, 
        t4.did
    from t4
    where 
        t4.tile = t4.tile8
        or t4.tile = t4.tile8 / 10
        or t4.tile = t4.tile8 / 100
        or t4.tile = t4.tile8 / 1000
        or t4.tile = t4.tile8 / 10000
        or t4.tile = t4.tile8 / 100000
        or t4.tile = t4.tile8 / 1000000
        or t4.tile = t4.tile8 / 10000000
    order by t4."day"
    )
"""

## If not

In [ ]:
query_did_osm_id = """
DROP TABLE IF EXISTS euro_stuttgart;
CREATE TABLE euro_stuttgart AS 

WITH unnested AS (
    SELECT 
        did, 
        unnest(stime_arr) AS stime, 
        unnest(tile_arr) AS tl8, 
        tile_id,
        day
    FROM (
        SELECT * 
        FROM ex_corona_sdkv6_2024_25
        WHERE tile_id IN (61075, 61076)
          AND day = '2024-06-19'
        UNION ALL
        SELECT * 
        FROM ex_corona_sdkv6_2024_27
        WHERE tile_id IN (61075, 61076)
        AND day = '2024-07-05'
    ) AS combined
),
geo AS (
    SELECT 
        un.did, 
        un.stime,
        un.day,
        ST_Transform(
            ST_Translate(
                ST_SetSRID(tile8togeo(un.tl8), 32632), 
                tx.minx, 
                tx.miny
            ), 
            3857
        ) AS geopoint
    FROM unnested AS un
    JOIN txc_dt_grid_1000m AS tx 
      ON un.tile_id = tx.tile_id
),
osm AS (
    SELECT osm_id, way
    FROM planet_osm_polygon
    WHERE osm_id = -9207449
)
SELECT DISTINCT geo.day, geo.did, osm.osm_id
FROM geo, osm
WHERE ST_Contains(osm.way, geo.geopoint);
"""


In [ ]:
query_osmid_did_geo = f"""
DROP TABLE IF EXISTS euro_stuttgart;
CREATE TABLE euro_stuttgart AS
WITH tiles AS (
    SELECT t.tile_id, t.plz
    FROM tile2plz t
    WHERE t.plz = '70372'
),
t1 AS (
    SELECT 
        ex.did,
        UNNEST(ex.stime_arr) AS stime,
        UNNEST(ex.tile_arr) AS tile8,
        ber.tile_id,
        ARRAY_LENGTH(ex.stime_arr, 1) AS stime_len
    FROM tiles ber
    JOIN (
        SELECT * FROM ex_corona_sdkv6_2024_25
        WHERE day = '2024-06-19'
        UNION ALL
        SELECT * FROM ex_corona_sdkv6_2024_27
        WHERE day = '2024-07-05'
    ) AS ex ON ber.tile_id = ex.tile_id
),
t2 AS (
    SELECT
        t1.did,
        t1.stime,
        t1.tile8,
        t1.tile_id,
        t1.stime_len,
        ST_Translate(ST_SetSRID(tile8togeo(t1.tile8), 32632), tx.minx, tx.miny) AS tilegeo
    FROM t1
    JOIN txc_dt_grid_1000m AS tx ON t1.tile_id = tx.tile_id
),
t3 AS (
    SELECT
        t2.did,
        t2.tile_id,
        t2.tile8,
        ST_X(ST_Transform(t2.tilegeo, 4326)) AS lon,
        ST_Y(ST_Transform(t2.tilegeo, 4326)) AS lat,
        t2.stime,
        pl.osm_id
    FROM t2
    JOIN planet_osm_polygon AS pl 
        ON ST_Contains(pl.way, t2.tilegeo)
    WHERE pl.osm_id = -9207449
)
SELECT
    did,
    ARRAY_AGG(tile8) AS tile8_ar,
    ARRAY_AGG(lon) AS lon_ar,
    ARRAY_AGG(lat) AS lat_ar,
    ARRAY_AGG(stime) AS stime_ar,
    osm_id
FROM t3
GROUP BY did, osm_id;
"""


In [ ]:
query_osmid_home = f"""
    with t1 as (
    	select
    		  vis."day"
    		, vis.did
    		, vis.osm_id
    		, hw.weighted_centroid as home
    	from euro_stuttgart as vis
    	join home_work_sdkv6 as hw
    	on hw.valid_for = (date_trunc('month', vis."day") + interval '1 month' - interval '1 day')::date
    	and hw.did = vis.did
    	where hw.place = 'home'
    )
    , t2 as (
    	select osm_id, way
    	from planet_osm_polygon
    	where osm_id = -9207449
    )
    select
          t1."day"
        , t1.did
        , st_x(st_transform(home, 4326)) as home_lon
        , st_y(st_transform(home, 4326)) as home_lat
        , t1.osm_id
        , st_distance(t1.home, st_transform(t2."way", 32632))/1000 as dist
    from t1
    join t2
    on t1.osm_id = t2.osm_id;
"""

In [ ]:
data = pd.DataFrame()

cursor.execute(query_did_osm_id)
conn_exe.commit()

data = pd.concat([data, pd.DataFrame(pd.read_sql_query(query_osmid_home, conn))])
#conn.commit()

In [ ]:
#data.to_csv('output/03_data_2024.csv', index=False)

In [ ]:
data

## plot visitor dids

In [ ]:
data

In [ ]:
import numpy as np
fig, ax = plt.subplots(figsize=[10,5])

data.dist.hist(ax=ax, bins=np.linspace(0,750,100))
#ax.set_yscale('log')
ax.set_xlabel('travel distance [km]', fontsize=15)
ax.set_ylabel('number unique visitors', fontsize=15)
ax.tick_params(labelsize=15)
ax.set_xlim([0,700])

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=[10,5])

data[data.dist<100].dist.hist(ax=ax, bins=np.linspace(0,100,50))
#ax.set_yscale('log')

plt.show()

In [ ]:
# Create a map centered at the first point
m = folium.Map(location=[48.7758, 9.1829], zoom_start=7)

# Add points to the map
for lat, lon in zip(data.home_lat, data.home_lon):
    folium.CircleMarker(
        location=[lat, lon], #popup=point["name"]
        radius=1,                  # Size of the dot
        color='blue',              # Outline color of the dot
        fill=True,                 # Fill the dot
        fill_color='blue',         # Fill color of the dot
        fill_opacity=0.7           # Transparency of the fill
    ).add_to(m)

folium.CircleMarker(
        location=[48.7758, 9.1829], #popup=point["name"]
        radius=1,                  # Size of the dot
        color='red',              # Outline color of the dot
        fill=True,                 # Fill the dot
        fill_color='red',         # Fill color of the dot
        fill_opacity=0.7           # Transparency of the fill
    ).add_to(m)

# Save the map to an HTML file (optional)
#m.save("plots/03_homeloc_distr_by_venue.html")

# Display the map in Jupyter Notebook (if using it)
m